# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# Current working directory:
print(os.getcwd())

# The path to event data folder:
filepath = os.getcwd() + '/event_data'

# Create a list of csv files and collect each filepath
file_path_list = glob.glob(os.path.join(filepath, '*.csv'))

/media/alena/Seagate Document/Documents/Udacity/data_engineer/code/project_data_modeling_cassandra/project-1b-project-template


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
     
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile:  
        csvreader = csv.reader(csvfile)  # new csv reader object
        next(csvreader)
        
        # extract each row one by one and append it to the data_rows_list        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# check the total number of rows 
print(len(full_data_rows_list))  # 8056

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

# create event_datafile_full.csv file (used to insert data into the 
# Apache Cassandra tables)
with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''): 
            continue        # skip rows with missing artist names
        # skip auth column (row[1])
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6],
                         row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in event_datafile_full.csv
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster()

# Create a session to establish connection and execute queries
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

>### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


>### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

>### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

$\color{blue}{\text{QUERY 1}}$

<div class="alert alert-info">
  <strong>The first query should output 3 fields (artist name, song title, and song length for a particular session id and the item within this session. Since the 'filter' fields here are session_id and item_in_session, these two fields would make the primary key. In particular, session_id would be a partition key, and item_in_session would be the clustering column. </strong>
</div>

In [8]:
# Query 1:  
# Give me the artist, song title and song's length in the music app history that was heard during \
# sessionId = 338, and itemInSession = 4

create_table = """
CREATE TABLE IF NOT EXISTS song_library (
    artist_name     TEXT, 
    song_title      TEXT, 
    song_length     FLOAT, 
    session_id      INT, 
    item_in_session INT, 
    PRIMARY KEY(session_id, item_in_session));
"""
try:
    session.execute(create_table)
except Exception as e:
    print(e)

In [9]:
file = 'event_datafile_new.csv'

In [10]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        # Insert data into the table
        query = "INSERT INTO song_library (session_id, item_in_session, artist_name, song_length, song_title)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], float(line[5]), line[9]))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
query = """
SELECT artist_name, song_title, song_length 
FROM song_library
WHERE session_id = 338 AND item_in_session = 4;
"""
df = pd.DataFrame()

try:
    rows = session.execute(query)
    for row in rows:
        df = df.append(pd.DataFrame(row).T).reset_index(drop=True)
except Exception as e:
    print(e)
    
# for row in rows:
#     print (row.artist_name, '|', row.song_title, '|', row.song_length)
    
df.columns = ['artist_name', 'song_title', 'song_length']
df    

,artist_name,song_title,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

$\color{blue}{\text{QUERY 2}}$

<div class="alert alert-info">
  <strong>The second query should output 4 fields (artist name, song title, and user's first and last names). It is stated that songs should be sorted by the item in session. Therefore, item_in_session should become a clustering column within the PRIMARY KEY. Since the 'filter' fields here are user_id and session_id, these two fields would complete the PRIMARY KEY,  specifiacally, they would become a composed partition key .</strong>
</div>

In [12]:
# Query 2: 
# Give me only the following: name of artist, song (sorted by itemInSession) 
# and user (first and last name) for userid = 10, sessionid = 182

create_table_query = """
CREATE TABLE IF NOT EXISTS user_session_history (
    artist_name TEXT,
    song_title       TEXT,
    user_fname       TEXT,
    user_lname       TEXT,
    user_id          INT,
    session_id       INT,
    item_in_session  INT,
    PRIMARY KEY((user_id, session_id), item_in_session));
"""
try:
    session.execute(create_table_query)
except Exception as e:
    print(e)
    
file = 'event_datafile_new.csv'  
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    # Insert data into the table
        query = "INSERT INTO user_session_history (user_id, session_id, item_in_session, artist_name, song_title, user_fname, user_lname)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

# Validate our data model -- verify the data was entered into the table.
df2 = pd.DataFrame()

query2 = """
SELECT artist_name, song_title, user_fname, user_lname 
FROM user_session_history
WHERE user_id = 10 AND session_id = 182;
""" 
try:
    rows = session.execute(query2)
    for row in rows:
        df2 = df2.append(pd.DataFrame(row).T).reset_index(drop=True)
except Exception as e:
    print(e)
    
# for row in rows:
#     print (row.artist_name, '|', row.song_title, '|', row.user_fname, row.user_lname) 
    
df2.columns = ['artist_name', 'song_title', 'user_fname', 'user_lname']
df2

,artist_name,song_title,user_fname,user_lname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


$\color{blue}{\text{QUERY 3}}$

<div class="alert alert-info">
  <strong>The third query only asks for user's first and last names. The only filtering condition is a particular song name. This sounds simple enough, although we should consider the possibility that some artists may name their songs equally. For example, both Amy Winehouse and Enrique Iglesias have a song named "Addicted". And since the query doesn't specify the artist name, I would assume that they would want to get that song name for all the artists, not only for the first one in the list. Thus I included artist_name into the table.
    Additionally, user_id would be a good user identifier to include into PRIMARY KEY along with the song_title. We need to include user_id to identify differnt users listening to the same songs. </strong>
</div>

In [13]:
# Query 3: 
# Give me every user name (first and last) in my music app history 
# who listened to the song 'All Hands Against His Own'

# Although queries to this table would equally work without artist name field, 
# I have included it to differenciate between the songs with the same names, but different artists 
# (for example, song _Addicted_ by Amy Winehouse vs _Addicted_ by Enrique Iglesias)
create_table_query = """
CREATE TABLE IF NOT EXISTS users_per_song_history (
    artist_name TEXT,
    song_title  TEXT,
    user_fname  TEXT,
    user_lname  TEXT,
    user_id     INT,
    PRIMARY KEY(song_title, user_id));
"""
try:
    session.execute(create_table_query)
except Exception as e:
    print(e)
    
file = 'event_datafile_new.csv'    
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    # Insert data into the table
        query = "INSERT INTO users_per_song_history (user_id, artist_name, song_title, user_fname, user_lname)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), line[0], line[9], line[1], line[4]))

# Validate our data model -- verify the data was entered into the table.
df3 = pd.DataFrame()

query3 = """
SELECT user_fname, user_lname 
FROM users_per_song_history
WHERE song_title = 'All Hands Against His Own';
"""       
try:
    rows = session.execute(query3)
    for row in rows:
        df3 = df3.append(pd.DataFrame(row).T).reset_index(drop=True)
except Exception as e:
    print(e)
    
# for row in rows:
#     print (row.user_fname, row.user_lname) 
    
df3.columns = ['user_fname', 'user_lname']
df3

,user_fname,user_lname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [14]:
query = "drop table song_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table user_session_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
        
query = "drop table users_per_song_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [15]:
session.shutdown()
cluster.shutdown()